## GSAT trend patterns

In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
import src.slurm_cluster as scluster
client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

### loaded the separated data from models

In [ ]:
dir1 ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure2/EC_Earth/'
EC_Earth_forced = xr.open_mfdataset(dir1 + 'GSAT_EC_Earth_Forced_anomalies_1850_2022.nc',chunks={'run': 1})
EC_Earth_internal = xr.open_mfdataset(dir1 + 'GSAT_EC_Earth_Internal_Variability_anomalies_1850_2022.nc', chunks={'run': 1})

In [ ]:
EC_Earth_internal

In [ ]:
EC_Earth_forced = EC_Earth_forced.rename({'__xarray_dataarray_variable__':'tas'})

In [ ]:
EC_Earth_forced

### Calculate the trend on 10-73yrs length with end year anchoring at 2022

In [ ]:
import pandas as pd
import numpy as np

start_year = 1950
end_year = 2022
min_length = 10

In [ ]:
years = np.arange(start_year, end_year + 1)

# Prepare a dictionary to hold the trend DataArrays for each period
trend_data = {}
p_value_data = {}

for begin_year in range(start_year, end_year - min_length+2):
    # Select the data from the start year to the last year
    time_slice = EC_Earth_forced.sel(year=slice(begin_year, end_year))
    
    # Calculate the trend for each grid cell
    trend, p_values = xr.apply_ufunc(
        func_mk,   # Pass the appropriate years
        time_slice,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    period_key = f'{begin_year}-{end_year}'
    trend_data[period_key] = trend
    p_value_data[period_key] = p_values

In [ ]:
trend_data

In [ ]:
# Convert dictionaries to xarray DataArray for saving
trend_da = xr.concat(trend_data.values(), dim=pd.Index(trend_data, name='period'))


In [ ]:
p_value_da = xr.concat(p_value_data.values(), dim=pd.Index(p_value_data, name='period'))

In [ ]:
trend_da['period']

In [ ]:
trend_da.sel(period='1950-2022')

In [ ]:
# trend_annual_np = {}
# pvalue_annual_np = {}

# for i in trend_data.keys():
#     trend_annual_np[i] = trend_data[i].values
#     pvalue_annual_np[i] = p_value_data[i].values

In [ ]:
# trend_annual_da = {}
# pvalue_annual_da = {}

# for interval, data in trend_annual_np.items():
#     trend_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": EC_Earth_forced_data_dict[interval].run, 
#     "lat": EC_Earth_forced_data_dict[interval].lat, "lon": EC_Earth_forced_data_dict[interval].lon})
# for interval, data in pvalue_annual_np.items():
#     pvalue_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": EC_Earth_forced_data_dict[interval].run, 
#     "lat": EC_Earth_forced_data_dict[interval].lat, "lon": EC_Earth_forced_data_dict[interval].lon})

In [ ]:
# out put the forced trend and pvalue
dir_out ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/LE_evaluation/Fig3_EC_Earth/output/'

trend_da.to_netcdf(dir_out + 'EC_Earth_forced_segmented_trend.nc')
# p_value_xr.to_netcdf(dir_out + 'EC_Earth_forced_segmented_pvalue.nc')

In [ ]:
trend_da